In [1]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CosmoDC2+RedMaPPer")

sys.path.insert(0, "/global/homes/c/cinlima/gcr-catalogs")
import GCRCatalogs

sys.path.insert(0, "global/homes/c/cinlima/clevar/clevar")

#Put nerc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

from clevar import ClCatalog, MemCatalog
from clevar.match import MembershipMatch, get_matched_pairs
from clevar.match import output_matched_catalog
from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf

import clevar

from richness_mass_calib import create_richness_mass_calib

In [2]:
from getdist import plots
from getdist.mcsamples import  MCSamples
import getdist
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))

import pygtc
import pandas as pd
from IPython.display import display, Math


GetDist Version: 1.4.5, Matplotlib version: 3.7.2


In [3]:
# #Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [4]:
# # Data
# DC2_halos_m200c = fits.open(
#     "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
# )
# dt_halos = Table(DC2_halos_m200c[1].data)
# # dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']


In [5]:
min_richness = 5
min_halo_mass = 1e13 #Msun

#Halos/Clusters
#SkySim + RedMaPPer: 'skysim5000_v1.1.1_redmapper_v0.8.5'
RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

#SkySim: 'skysim5000_v1.1.1_image'
halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

#Members
RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

#Match catalogs
mt = MembershipMatch()

mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/SS_RM_match_cross_membership')

RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

clusters_dt = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('m200c', 'richness', 'redshift_true'))

dt_halos = clusters_dt


RM_cat
    * ClEvar used in matching: 0.14.1
 * Total objects:    46,491
 * multiple (self):  3,921
 * multiple (other): 3,921
 * unique (self):    3,920
 * unique (other):   3,920
 * cross:            3,920

halo_cat
    * ClEvar used in matching: 0.14.1
 * Total objects:    249,495
 * multiple (self):  9,756
 * multiple (other): 9,756
 * unique (self):    3,920
 * unique (other):   3,920
 * cross:            3,920


In [6]:
ascaso = Nc.ClusterMassAscaso()
fixed_parameters_ascaso = [6] #fixing cut parameter

ascaso.param_set_by_name("cut", 1e15) #Set cut parameter value 
mset = Ncm.MSet()

lnrich_ext = Nc.ClusterMassLnrichExt()

def catalog_fit(dt_halos, ascaso = True):
    rmdata = create_richness_mass_calib(dt_halos)
    dset = Ncm.Dataset.new()
    dset.append_data(rmdata)

    lh = Ncm.Likelihood.new(dset)
    
    if ascaso:     
        mset.set(ascaso)
        rmdata.m2lnL_val(mset)  
        mset.param_set_all_ftype(Ncm.ParamType.FREE) #All parameters free

        #All parameters free except cut parameters:
        for par in fixed_parameters_ascaso:
            mset.param_set_ftype(7000, par, Ncm.ParamType.FIXED)

        mset.prepare_fparam_map()

    else:
        fixed_parameters_lnrich_ext = [ 13, 14, 15] #fixing cut parameters

        lnrich_ext.param_set_by_name("cut", 1e15) #Set cut parameter value 
        mset.set(lnrich_ext)
        rmdata.m2lnL_val(mset)  
        mset.param_set_all_ftype(Ncm.ParamType.FREE) #All parameters free

        #All parameters free except cut parameters:
        for par in fixed_parameters_lnrich_ext:
            mset.param_set_ftype(7000, par, Ncm.ParamType.FIXED)

        mset.prepare_fparam_map()

    fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )
    fit.log_info()
    fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)
    fit.log_info()
        
    return fit

fit = catalog_fit(dt_halos, ascaso=False)

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.19                [FREE]
#   -    muM1[01]:  0.868588963806504   [FREE]
#   -    muZ1[02]: -0.304006137332276   [FREE]
#   -    muM2[03]:  0                   [FREE]
#   -    muZ2[04]:  0                   [FREE]
#   -    muMZ[05]:  0                   [FREE]
#   -  sigma0[06]:  0.33                [FREE]
#   - sigmaM1[07]: -0.0347435585522601  [FREE]
#   - sigmaZ1[08]:  0                   [FREE]
#   - sigmaM2[09]:  0                   [FREE]
#   - sigmaZ2[10]:  0                   [FREE]
#   - sigmaMZ[11]:  0                   [FREE]
#   -      A0[12]:  0                   [FRE

In [7]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.01494803696517    [FREE]
#   -    muM1[01]:  0.759730515721659   [FREE]
#   -    muZ1[02]:  0.264503842640676   [FREE]
#   -    muM2[03]:  0.167202753143723   [FREE]
#   -    muZ2[04]: -0.136277325376878   [FREE]
#   -    muMZ[05]:  0.10284492332365    [FREE]
#   -  sigma0[06]:  0.000102836566421796 [FREE]
#   - sigmaM1[07]: -0.336223480051037   [FREE]
#   - sigmaZ1[08]: -0.236350637482794   [FREE]
#   - sigmaM2[09]: -0.0675238673159113  [FREE]
#   - sigmaZ2[10]: -0.0175501645389768  [FREE]
#   - sigmaMZ[11]: -0.128577050860081   [FREE]
#   -      A0[12]: -0.493594590612998   [FR

In [8]:
nwalkers = 1500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("ascaso1.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 1500.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.02003970539531    [FREE]
#   -    muM1[01

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



     -0.166153280901276     -0.356850306111385
# NcmFitESMCMC: Catalog row 1494169: m2lnL =    -531.041465495243, recalculated to    -531.041465495243, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 1494169 values:     -531.041465495243       4.00423128472553      0.748451412658282      0.313919037351235      0.163706464080655    -0.0717660549853002      0.141821345055412     0.0140661878918941     -0.361678872621871      -0.25271198886503    -0.0707274998922714   -0.00258548022497243     -0.130935390688514     -0.463878247483437
# NcmFitESMCMC: Catalog row 1494170: m2lnL =     -512.89479448181, recalculated to     -512.89479448181, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 1494170 values:      -512.89479448181       4.01876362313272      0.781585622389422      0.102179455735472       0.17403807262296     0.0263279550818918    -0.0184170533092114     0.0928176004121823     -0.356367729321353     -0.251755626455021     -0.069843783500263       0.23260200403404     -0.143094

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



# NcmFitESMCMC: Catalog row 1494314: m2lnL =    -516.581259832323, recalculated to    -516.581259832323, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 1494314 values:     -516.581259832323       4.02562805092787      0.755855934385169      0.192504567206407      0.167999675248069      -0.22157033066755     0.0662146998081484      0.032739589679796     -0.379119263011345     -0.299061789613631    -0.0810389683455881      -0.14034162020569     -0.183617638385526     -0.449651338112273
# NcmFitESMCMC: Catalog row 1494315: m2lnL =    -516.411703210518, recalculated to    -516.411703210518, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 1494315 values:     -516.411703210518       4.02175162075192         0.777062959501      0.122699334376252      0.172738995405653    -0.0779975449234235    0.00915006905082175      0.136401571938637     -0.424319035614882     -0.234396755749232    -0.0850553224287825      0.366207351835083     -0.109985023536871     -0.341799937039917
# NcmFitESMCMC

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



    -0.129935111812977     -0.383562988331141
# NcmFitESMCMC: Catalog row 1494692: m2lnL =     -515.43961781438, recalculated to     -515.43961781438, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 1494692 values:      -515.43961781438       4.01961663711862      0.789757966585807      0.381879613983598      0.179447164985841      0.159805692218895      0.148039648471617     0.0153141039992451     -0.345703153910544     -0.350199051350363    -0.0700105208208982    -0.0247813039720143     -0.218707459252039     -0.476611563789739
# NcmFitESMCMC: Catalog row 1494693: m2lnL =    -515.537176766224, recalculated to    -515.537176766224, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 1494693 values:     -515.537176766224       4.02544485753226      0.782161693759067      0.398302200788187      0.174801745146702      0.211598227913415      0.201279645825367     0.0447841127671532     -0.297466977504272     -0.190282725377145    -0.0568664198606462      0.190175294984026     -0.1065757

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  0.051331     0.066553   
# NcmMSetCatalog: Current var:    41288        0.00038205   0.00055136   0.0047226    7.4434e-05   0.024172     0.0025783    0.0056027    0.001838     0.0047004    0.00011106   0.030628     0.0026349    0.0044293  
# NcmMSetCatalog: Current tau:    1            1            1            1            1            1            1            1            1            1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  1.0001       1.0107       1.0106       1.0108       1.01         1.0112       1.0108       1.0266       1.0224       1.0141       1.0163       1.0125       1.0138       1.0241     
# NcmMSetCatalog: Maximal Shrink factor = 1.04194               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `998':
# NcmMSetCatalog: Constant break point at `74':
# NcmFitESMCMC:acceptance ratio 11.4000%

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [9]:
esmcmc.mean_covar()
fit.log_covar()

#----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#      mu[07000:00] =  4.015       +/-  0.01954     |  1           |  0.803       |  0.1332      |  0.5465      | -0.1924      |  0.1057      | -0.03568     |  0.002822    |  0.008087    | -0.02797     | -0.001361    |  0.002811    | -0.03063     |
#    muM1[07000:01] =  0.7597      +/-  0.02348     |  0.803       |  1           |  0.2512      |  0.8789      |  0.008235    |  0.2448      | -0.04269     | -0.001788    |  0.02417     | -0.03169     |  0.01754     |  0.02174     | -0.03518     |
#    muZ1[07000:02] =  0.2654      +/-  0.06872     |  0.1332      |  0.2512      |  1           |  0.2935      |  0.1397      |  

In [10]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.01529095619252    [FREE]
#   -    muM1[01]:  0.759738982800987   [FREE]
#   -    muZ1[02]:  0.265393102729109   [FREE]
#   -    muM2[03]:  0.167098952309787   [FREE]
#   -    muZ2[04]: -0.136577268867969   [FREE]
#   -    muMZ[05]:  0.103871658746802   [FREE]
#   -  sigma0[06]:  0.0823387357504222  [FREE]
#   - sigmaM1[07]: -0.376165690519166   [FREE]
#   - sigmaZ1[08]: -0.265991501051027   [FREE]
#   - sigmaM2[09]: -0.0760695479568851  [FREE]
#   - sigmaZ2[10]: -0.0309761329895628  [FREE]
#   - sigmaMZ[11]: -0.146635768205235   [FREE]
#   -      A0[12]: -0.415015257364698   [FRE

In [11]:
data_fit_full = pd.DataFrame(fits.open("ascaso1.fits")[1].data).iloc[:, 1:14].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    '10',
    '11',
    '12',
    '13'
]
labels=["\mu_0", "\mu_{M1}", "\mu_{M2}", "\mu_{Z1}"," \mu_{Z2}", "\mu_{MZ}", "\sigma_0", "\sigma_{M1}", "\sigma_{M2}", "\sigma_{Z1}", "\sigma_{Z2}", "\sigma_{MZ}","A_0" ]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples2 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples2.removeBurn(0.3)


Removed no burn in


In [12]:
# Triangle plot
g2 = plots.get_subplot_plotter()

g2.settings.lab_fontsize = 40
g2.settings.legend_fontsize = 40
g2.settings.axes_fontsize = 30

g2.triangle_plot(
    [samples2],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fine_bins=1,
    colors=['#b186f1'],
    fontsize=70,
    line_args=[{'lw':1.2,'ls':'-', 'color':'#b186f1'}],
    legend_labels=["Estendido", ],
)
#plt.savefig('mcmc.png')
plt.show()


In [13]:
for i in range(1, 14):
    display(Math(samples2.getInlineLatex(str(i),limit=1)))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [14]:
min_richness = 5
min_halo_mass = 1e13 #Msun

#Halos/Clusters
#CosmoDC2 + RedMaPPer: 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

#CosmoDC2: 'cosmoDC2_v1.1.4_image'
halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

#Members
RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

#Match catalogs
mt = MembershipMatch()

mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/cosmoDC2_RM_match_cross_membership')

#Matched Data
RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

cdc2_dt = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('m200c', 'richness', 'redshift_true'))
dt_halos = cdc2_dt


fit = catalog_fit(dt_halos, ascaso=False)


RM_cat
    * ClEvar used in matching: 0.14.1
 * Total objects:    45,081
 * multiple (self):  43,213
 * multiple (other): 43,213
 * unique (self):    43,035
 * unique (other):   43,037
 * cross:            43,035

halo_cat
    * ClEvar used in matching: 0.14.1
 * Total objects:    471,457
 * multiple (self):  75,785
 * multiple (other): 75,785
 * unique (self):    43,037
 * unique (other):   43,035
 * cross:            43,035
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.01529095619252    [FREE]
#   -    muM1[01]:  0.759738982800987   [FREE]
#   -    muZ1[02]:  0.265393102729109   [FREE]
#   -    muM2[03]:  0.167098952309787

In [15]:
nwalkers = 1500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("mcmc2.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()


# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 1500.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.14193788519047    [FREE]
#   -    muM1[01

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



#  Elapsed time: 00 days, 00:00:01.2727090
#  iteration            [000297]
#  function evaluations [000299]
#  gradient evaluations [000000]
#  degrees of freedom   [043022]
#  m2lnL     =     5178.40910025064 (     5165.3434 )
#  Fit parameters:
#     4.1422295599719        1.37481493142788       0.329486751121546      0.207103155456032     -0.829788354281093      0.137805585885355      0.525395570781671     -0.224179452484652     -0.865248050708878      0.167105245040748      1.26243678089361      -0.482683119251618      0.167615989981289     
# NcmMSetCatalog: Current mean:   5262.2       4.1394       1.3729       0.32758      0.20686     -0.83602      0.13598      0.52384     -0.22824     -0.87058      0.16455      1.2609      -0.48256      0.16701    
# NcmMSetCatalog: Current msd:    1.4354       4.4806e-05   3.2128e-05   6.0758e-05   6.4755e-06   8.2709e-05   2.4739e-05   5.5731e-05   6.821e-05    0.00015755   3.7218e-05   0.00032527   9.0129e-05   1.4279e-05 
# NcmMSetCatalog:

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



#  Elapsed time: 00 days, 00:00:01.2727090
#  iteration            [000297]
#  function evaluations [000299]
#  gradient evaluations [000000]
#  degrees of freedom   [043022]
#  m2lnL     =     5178.24363383903 (     5165.3434 )
#  Fit parameters:
#     4.14180013091141       1.37447112046046       0.329426861963978      0.207038588681094     -0.828125325806845      0.137800947370078      0.524226421073657     -0.225764829482537     -0.865044033047151      0.165925203436632      1.26067253073339      -0.483838471234505      0.167230581340196     
# NcmMSetCatalog: Current mean:   5247.8       4.1398       1.3732       0.32795      0.20689     -0.83523      0.1363       0.52386     -0.22787     -0.87026      0.16478      1.2612      -0.48306      0.16705    
# NcmMSetCatalog: Current msd:    1.1898       3.8091e-05   2.7497e-05   5.4334e-05   5.5339e-06   7.5155e-05   2.185e-05    5.067e-05    6.2034e-05   0.00014666   3.3179e-05   0.00030777   8.2756e-05   1.2409e-05 
# NcmMSetCatalog:

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



#  Elapsed time: 00 days, 00:00:01.2727090
#  iteration            [000297]
#  function evaluations [000299]
#  gradient evaluations [000000]
#  degrees of freedom   [043022]
#  m2lnL     =     5178.31466489895 (     5165.3434 )
#  Fit parameters:
#     4.14079551327538       1.37357398085676       0.328300998818854      0.206859628807425     -0.831829032166556      0.137373032945246      0.524997190170281     -0.225333953107072     -0.863388374996304      0.166172786187474      1.25072827758583      -0.482796721212707      0.167322712806547     
# NcmMSetCatalog: Current mean:   5209.3       4.1407       1.3737       0.32904      0.20694     -0.83251      0.13721      0.52424     -0.22647     -0.86896      0.16562      1.259       -0.4843       0.16724    
# NcmMSetCatalog: Current msd:    0.53115      1.9812e-05   1.4788e-05   3.4456e-05   2.9578e-06   4.9776e-05   1.3289e-05   3.3807e-05   4.138e-05    0.00010267   2.085e-05    0.00022592   5.6366e-05   7.1175e-06 
# NcmMSetCatalog:

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  0.0078696  
# NcmMSetCatalog: Current var:    3.2302e+05   0.00047085   0.00026456   0.0014814    1.0568e-05   0.0031047    0.00021884   0.0014341    0.0021458    0.013315     0.00053979   0.064832     0.0039964    6.193e-05  
# NcmMSetCatalog: Current tau:    1            1            1            1            1            1            1            1            1            1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  1.0001       1.0107       1.0108       1.0086       1.0106       1.0093       1.009        1.0095       1.0095       1.0097       1.0106       1.0089       1.0097       1.0118     
# NcmMSetCatalog: Maximal Shrink factor = 1.01299               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `918':
# NcmMSetCatalog: Constant break point at `111':
# NcmFitESMCMC:acceptance ratio 12.5149% (last updat

In [16]:
fit.log_covar()

#----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#      mu[07000:00] =  4.141       +/-  0.02126     |  1           |  0.9216      | -0.03785     |  0.7972      | -0.06383     | -0.02458     | -0.1976      | -0.2408      |  0.04788     | -0.2289      | -0.004194    |  0.001432    | -0.1816      |
#    muM1[07000:01] =  1.374       +/-  0.016       |  0.9216      |  1           |  1.732e-06   |  0.9552      |  0.006245    | -0.003124    | -0.2095      | -0.2635      |  0.04598     | -0.2442      | -0.002564    |  0.0006881   | -0.1837      |
#    muZ1[07000:02] =  0.3291      +/-  0.03833     | -0.03785     |  1.732e-06   |  1           |  0.032       | -0.08255     |  

In [17]:
data_fit_full = pd.DataFrame(fits.open("mcmc2.fits")[1].data).iloc[:, 1:14].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    '10',
    '11',
    '12',
    '13'
]
labels=["\mu_0", "\mu_{M1}", "\mu_{M2}", "\mu_{Z1}"," \mu_{Z2}", "\mu_{MZ}", "\sigma_0", "\sigma_{M1}", "\sigma_{M2}", "\sigma_{Z1}", "\sigma_{Z2}", "\sigma_{MZ}","A_0" ]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples3 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples3.removeBurn(0.3)

Removed no burn in


In [18]:
# Triangle plot
g2 = plots.get_subplot_plotter()

g2.settings.lab_fontsize = 40
g2.settings.legend_fontsize = 40
g2.settings.axes_fontsize = 30

g2.triangle_plot(
    [samples3],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fine_bins=1,
    colors=['#b186f1'],
    fontsize=70,
    line_args=[{'lw':1.2,'ls':'-', 'color':'#b186f1'}],
    legend_labels=["Estendido", ],
)
plt.savefig('mcmc2.png')
plt.show()


In [ ]:
for i in range(1, 14):
    display(Math(samples3.getInlineLatex(str(i),limit=1)))
